# CMPINF Final Project
## Evan Tall, Jerry Huang, Hailey Trampel

## How we ranked 'bestness':
* How **safe** the neighborhood is
* How many **hospitals** the neighborhood has
* How many **bridges** the neighborhood has 

In [1]:
import pandas as pd
import numpy as np
import ssl

## Analyzing how safe the neighborhoods are
First, we imported the data for Pittsburgh Area arrests, as reported by the Pittsburgh Police.

In [6]:
pgh_crime_data = pd.read_csv("crime_data.csv")
pgh_crime_data.head()

,PK,CCR,AGE,GENDER,RACE,ARRESTTIME,ARRESTLOCATION,OFFENSES,INCIDENTLOCATION,INCIDENTNEIGHBORHOOD,INCIDENTZONE,INCIDENTTRACT,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y
0,1975272,16158872,42.0,F,B,2016-08-24T12:20:00,"4700 Block Centre AV Pittsburgh, PA 15213",3929 Retail Theft.,"4700 Block Centre AV Pittsburgh, PA 15213",Bloomfield,5,804.0,8.0,2.0,-79.949277,40.452551
1,1974456,16144120,31.0,M,W,2016-08-03T14:55:00,"4200 Block Steubenville PKE Pittsburgh, PA 15205",13(a)(16) Possession of Controlled Substance,"4200 Block Steubenville PKE Pittsburgh, PA 15205",Outside City,OSC,5599.0,NaN,NaN,-80.088018,40.440136
2,1974466,16144165,63.0,F,B,2016-08-03T16:45:00,"900 Block Freeport RD Fox Chapel, PA 15238",3929 Retail Theft.,"900 Block Freeport RD Fox Chapel, PA 15238",Westwood,5,2811.0,9.0,2.0,-79.891803,40.486625
3,1974550,16145257,25.0,F,W,2016-08-05T02:36:00,"Foreland ST & Cedar AV Pittsburgh, PA 15212",5503 Disorderly Conduct. / 5505 Public Drunken...,"Foreland ST & Cedar AV Pittsburgh, PA 15212",East Allegheny,1,2304.0,1.0,1.0,-80.001939,40.454080
4,1974596,16145962,25.0,M,B,2016-08-06T02:00:00,"900 Block Woodlow ST Pittsburgh, PA 15205",2702 Aggravated Assault. / 2705 Recklessy Enda...,"900 Block Woodlow ST Pittsburgh, PA 15205",Crafton Heights,5,2814.0,2.0,5.0,-80.052204,40.445900


In [9]:
pgh_safety_data = pd.read_csv("public_safety_data.csv")
pgh_safety_data.head()

,Neighborhood,Sector #,Population (2010),#Part 1 (Major Crime) Reports (2010),#Part 2 Reports (2010),#Other Police Reports (2010),Part 1 Crime per 100 Persons (2010),Part 2 Crime per 100 Persons (2010),#Murder (2010),#Rape (2010),#Robbery (2010),#Agr. Assault (2010),#Burglary (2010),#Auto Theft (2010),#Drug Violations (2010)
0,Allegheny Center,3,933,71,141,46,7.6,15.1,0,2,9,5,8,5,31
1,Allegheny West,3,462,38,49,31,8.2,10.6,1,0,4,6,6,4,12
2,Allentown,6,"2,500",159,252,160,6.4,10.1,0,2,15,23,48,11,66
3,Arlington,7,"1,869",50,84,48,2.7,4.5,0,1,8,5,15,7,9
4,Arlington Heights,7,244,19,37,24,7.8,15.2,0,0,4,5,5,3,7


## Counting the number of hospitals per region

In [7]:
pgh_bridge_data = pd.read_csv("bridge_data.csv")
pgh_bridge_data.head()

,id,name,start_neighborhood,end_neighborhood,year_built,year_rehab,image,neighborhood,council_district,ward,tract,public_works_division,pli_division,police_zone,fire_zone,latitude,longitude
0,2002832442,10th St Bypass,Central Business District,NaN,NaN,NaN,https://tools.wprdc.org/images/pittsburgh/brid...,Central Business District,6,2.0,42003020100,6.0,2.0,2.0,1-3,40.446442,-79.996534
1,761156487,28th Street Bridge,Strip District,Polish Hill,1931.0,1974.0,https://tools.wprdc.org/images/pittsburgh/brid...,Strip District,7,6.0,42003060500,6.0,6.0,2.0,2-25,40.456172,-79.973708
2,290122944,2nd Avenue Bridge,Squirrel Hill South,Swisshelm Park,1886.0,1970.0,https://tools.wprdc.org/images/pittsburgh/brid...,Swisshelm Park,5,14.0,42003141400,3.0,14.0,4.0,2-12,40.415202,-79.915695
3,1811170969,Alexander Street Bridge,West End,NaN,1924.0,2000.0,https://tools.wprdc.org/images/pittsburgh/brid...,West End,2,20.0,42003562600,5.0,20.0,6.0,1-17,40.440072,-80.033365
4,2047190591,Allegheny Channel Boardwalk,Troy Hill,NaN,NaN,NaN,https://tools.wprdc.org/images/pittsburgh/brid...,Troy Hill,1,24.0,42003240600,1.0,24.0,1.0,1-1,40.466578,-79.977728
